### This script converts the RP Questionnaire to Word

In [2]:
import pandas as pd
from docx import Document

### Input and Output files

In [18]:
input_excel_file = "../docs/draft_RP_questionnaire_2024_09_09.xlsx"
output_word_file = "../docs/Survey_Documentation.docx"

### Load data as dataframes

In [19]:
# Load both sheets
df_main = pd.read_excel(input_excel_file, sheet_name="Questions")
df_options = pd.read_excel(input_excel_file, sheet_name="Response Options")
df_modes = pd.read_excel(input_excel_file, sheet_name = "mode")

### Method to add documentation of a segment (employee, arriving passenger, departing passenger)

In [1]:
def add_segment_documentation(doc, segment):
    # Iterate through each row in the main DataFrame
    doc.add_heading(segment, level = 2)
    for index, row in df_main.iterrows():
        # Skip rows where 'Departing passenger' is blank or "N/A"
        if pd.isna(row[segment]) or row[segment] == "N/A":
            continue
        # Create the variable name heading (this will be trackable in Word)
        variable_heading = f"{int(row['Question number'])}. {row['Variable name']}"
        heading = doc.add_heading(variable_heading, level=3)

        # Remove bold formatting from the heading
        run = heading.runs[0]
        run.bold = False

        # Add question category and question text
        doc.add_paragraph(f"Question Category: {row['Question category']}")
        doc.add_paragraph(f"Question: {row[segment]}")

        # Add the "Response Options" title
        doc.add_paragraph("Response Options:")

        # Filter the options DataFrame for the current variable
        options = df_options[df_options['Variable'] == row['Variable name']]
        
        # Check for common response options from the df_modes grid
        if row['Variable name'] in df_modes.columns:
            # Find the response options for the current mode in df_modes
            mode_options = df_modes[df_modes[row['Variable name']] == 1]

        # If there are options, add them as a table
        if not options.empty:
            table = doc.add_table(rows=1, cols=2)
            table.autofit = True
            table.style = 'Table Grid'
            # Add header cells
            hdr_cells = table.rows[0].cells
            hdr_cells[0].text = 'Codes'
            hdr_cells[1].text = 'Labels'

            # Add data rows
            for _, option_row in options.iterrows():
                row_cells = table.add_row().cells
                row_cells[0].text = str(option_row['Codes'])
                row_cells[1].text = option_row['Labels']
        else:
            doc.paragraphs[-1].add_run(" Set of options not applicable")
        # Add a space between questions
        doc.add_paragraph("\n")

### Iterating over all segments:

In [22]:
# Create a new Word Document
doc = Document()
doc.add_heading("ATC 2024 Survey Documentation", level = 1)

#segments to iterate over
segments = ['Departing passenger', 'Arriving passenger', 'Employee']

for segment in segments:
    add_segment_documentation(doc, segment)

### Writing the document

In [23]:
# Save the document
doc.save(output_word_file)
print(f"Document saved as {output_word_file}")

Document saved as ../docs/Survey_Documentation.docx
